In [3]:
import pandas as pd
import numpy as np
from pydantic import BaseModel, confloat
from typing import List, Tuple

In [4]:
class AntColonyModelInput(BaseModel):
    points: List[Tuple[float, float]]

class AntColonyModelOutput(BaseModel):
    path: List[Tuple[float, float]]
    distance: float

In [5]:
class AntColonyModelParams(BaseModel):
    alpha: float = 3.0
    beta: float = 2.0
    evaporation_rate: confloat(gt=0, lt=1) = 0.5
    initial_pheromone: float = 1.0    

In [6]:
class AntTrail(BaseModel):
    ant_trail_points: List[Tuple[float, float]]
    distance: float

class AntsTrails(BaseModel):
    trails: List[AntTrail]

In [110]:
class AntColonyModel:
    def __init__(self, input: AntColonyModelInput):
        self.model_params = AntColonyModelParams()
        self.points: List[Tuple[float, float]] = input.points
        self.n_points = len(self.points)
        self.distance_matrix = self._build_distance_matrix()
        self.mutable_pheromone_matrix = np.full((self.n_points, self.n_points), fill_value=self.model_params.initial_pheromone)

    def _build_distance_matrix(self) -> np.ndarray:
        points = np.array(self.points)
        print(points)
        matrix = np.zeros((self.n_points, self.n_points))

        for i in range(self.n_points):
            for j in range(self.n_points):
                if i != j:
                    matrix[i, j] = np.linalg.norm(points[i] - points[j])
        
        return matrix

    def _get_path_probability(self, distance: float, pheromone: float, pheromone_density: float) -> float:
        individual_pheromone_density = np.power(pheromone, self.model_params.alpha) / np.power(distance, self.model_params.beta)
        probability = individual_pheromone_density / pheromone_density
        return probability

    def _get_pheromone_density_list(self, distance_matrix: np.ndarray) -> float:
        values = []
        for i in range(self.n_points):
            value = 0
            for j in range(self.n_points):
                if i != j:
                    value += np.power(self.mutable_pheromone_matrix[i, j], self.model_params.alpha) / np.power(distance_matrix[i, j], self.model_params.beta)
            values.append(value)

        return values


    def _build_probability_matrix(self, distance_matrix: np.ndarray) -> np.ndarray:
        matrix = np.zeros((self.n_points, self.n_points))
        pheromone_density_list = self._get_pheromone_density_list(distance_matrix)
        for i in range(self.n_points):
            pheromone_density = pheromone_density_list[i]
            for j in range(self.n_points):
                if i != j:
                    distance = distance_matrix[i, j]
                    pheromone = self.mutable_pheromone_matrix[i, j]
                    matrix[i, j] = self._get_path_probability(distance, pheromone, pheromone_density)
        
        return matrix

    def _get_tail_points(self, chosen_order: List[int]) -> List[Tuple[float, float]]:
        points = []
        for i in chosen_order:
            points.append(self.points[i])
        return points

    def _get_trail(self, ant_probable_trail: np.ndarray) -> List[int]:
        valid_indices = np.where(ant_probable_trail > 0)[0]
        valid_probs = ant_probable_trail[valid_indices].astype(float)

        chosen_order = []
        while len(valid_indices) > 0:
            valid_probs = valid_probs / valid_probs.sum()
            idx = np.random.choice(len(valid_indices), p=valid_probs)
            chosen_order.append(valid_indices[idx])

            valid_indices = np.delete(valid_indices, idx)
            valid_probs = np.delete(valid_probs, idx)

        return chosen_order

           
    def _get_distance(self, ant_trail_distances: np.ndarray, ant_trail: List[int]) -> float:
        distance = 0
        for i in range(len(ant_trail)):
            distance += ant_trail_distances[ant_trail[i]]
        return distance

    def _get_ants_trails(self, probability_matrix: np.ndarray) -> AntsTrails:
        trails = []
        for i in range(self.n_points):
            ant_probable_trail = probability_matrix[i]
            ant_trail = self._get_trail(ant_probable_trail)
            ant_trail_points = self._get_tail_points(ant_trail)
            ant_trail_distances = self.distance_matrix[i]
            distance = self._get_distance(ant_trail_distances, ant_trail)
            trails.append(AntTrail(ant_trail_points=ant_trail_points, distance=distance))
        return AntsTrails(trails=trails)

    def run(self):
        probability_matrix = self._build_probability_matrix(self.distance_matrix)
        ants_trails: AntsTrails = self._get_ants_trails(probability_matrix)

        return ants_trails
        

In [111]:
points = [(2, 5), (4, 3), (6, 8), (1, 2)]
input = AntColonyModelInput(points=points)

model = AntColonyModel(input=input)

result = model.run()
print(result)

[[2. 5.]
 [4. 3.]
 [6. 8.]
 [1. 2.]]
trails=[AntTrail(ant_trail_points=[(6.0, 8.0), (4.0, 3.0), (1.0, 2.0)], distance=10.99070478491457), AntTrail(ant_trail_points=[(1.0, 2.0), (2.0, 5.0), (6.0, 8.0)], distance=11.375869592049074), AntTrail(ant_trail_points=[(4.0, 3.0), (2.0, 5.0), (1.0, 2.0)], distance=18.19541448304116), AntTrail(ant_trail_points=[(2.0, 5.0), (4.0, 3.0), (6.0, 8.0)], distance=14.134804996243414)]


In [9]:
point1 = np.array((2, 5))
point2 = np.array((4, 3))
point3 = np.array((6, 8))
point4 = np.array((1, 2))

distance1 = np.linalg.norm(point2 - point4)
print(distance1)
distance2 = np.linalg.norm(point4 - point3)
print(distance2)

3.1622776601683795
7.810249675906654


In [84]:
import numpy as np

probs = np.array([0.0, 0.47169811, 0.0, 0.1509434, 0.37735849])

# Get valid indices (skip zeros)
valid_indices = np.where(probs > 0)[0]
valid_probs = probs[valid_indices].astype(float)

chosen_order = []

while len(valid_indices) > 0:
    # Normalize current probabilities
    valid_probs = valid_probs / valid_probs.sum()
    
    # Pick one index according to probabilities
    idx = np.random.choice(len(valid_indices), p=valid_probs)
    
    # Save the original index
    chosen_order.append(valid_indices[idx])
    
    # Remove the chosen one
    valid_indices = np.delete(valid_indices, idx)
    valid_probs = np.delete(valid_probs, idx)

print("Chosen order:", chosen_order)


Chosen order: [np.int64(1), np.int64(4), np.int64(3)]


In [90]:
for i in range(3 - 1):
    print(i)

0
1
